In [1]:
import os
import json

from pymongo import MongoClient

Set environment variables and setup simulated vasp run

In [2]:
#set the path to vasp pseudopotential files
#os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "reference_files")

# Structure and input set

In [3]:
from pymatgen import Structure

struct_si = Structure([[5.468344455957462, 0.0, -0.0], 
                       [-0.0, 5.468344455957462, 0.0], 
                       [0.0, -0.0, 5.468344455957462]], 
                      
                      ["Si"]*8, 
                      
                      [[-0.0, -0.0, 0.5],
                       [0.75, 0.75, 0.75],
                       [0.0, 0.5, -0.0],
                       [0.75, 0.25, 0.25],
                       [0.5, 0.0, -0.0],
                       [0.25, 0.75, 0.25],
                       [0.5, 0.5, 0.5],
                       [0.25, 0.25, 0.75]])

In [4]:
from pymatgen.io.vasp.sets import MPRelaxSet, DictSet

vis = MPRelaxSet(struct_si, force_gamma=True)
vis.config_dict["KPOINTS"].update({"reciprocal_density": 600})
vis = DictSet(struct_si, vis.config_dict)

# Structure optimization firework

In [5]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW, StaticFW, TransmuterFW


vasp_cmd=None
db_file=">>db_file<<"

fws = []
fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=vasp_cmd, db_file=db_file)
fws.append(fw1)

# Deformation fireworks

In [6]:
from pymatgen.analysis.elasticity.strain import Deformation
from fireworks import Firework
from matmethods.vasp.workflows.base.elastic import PassStressStrainData

norm_deformations=[-0.01, -0.005, 0.005, 0.01]
shear_deformations=[-0.06, -0.03, 0.03, 0.06]

deformations = []
#Normal deformations
for ind in [(0, 0), (1, 1), (2, 2)]:
    for amount in norm_deformations:
        defo = Deformation.from_index_amount(ind, amount)
        deformations.append(defo)
        
#Shear deformations
for ind in [(0, 1), (0, 2), (1, 2)]:
    for amount in shear_deformations:
        defo = Deformation.from_index_amount(ind, amount)
        deformations.append(defo)

#update incar file settings
def_incar_settings = vis.incar.as_dict()
def_incar_settings.update({"ISIF":2, "ISTART":1})
for key in ["MAGMOM", "@module", "@class", "LDAUU", "LDAUJ", "LDAUL"]:
    def_incar_settings.pop(key, None)

def_vasp_params = {"user_incar_settings":def_incar_settings}
def_vasp_params.update({"reciprocal_density": 600})

#generate deformation fireworks; one for each deformation
for i, deformation in enumerate(deformations):
    fw = TransmuterFW(name="elastic_deformation_"+str(i+1),
                      structure=struct_si,
                      transformations=['DeformStructureTransformation'],
                      transformation_params=[{"deformation": deformation.tolist()}],
                      copy_vasp_outputs=True,
                      db_file=db_file,
                      vasp_cmd=vasp_cmd,
                      parents=fws[0],
                      vasp_input_params=def_vasp_params
                     )
    fw.spec['_tasks'].append(PassStressStrainData(deformation=deformation.tolist()).to_dict())
    fws.append(fw)

# Stress-strain analyzer firework

In [7]:
from matmethods.vasp.workflows.base.elastic import AnalyzeStressStrainData

fws.append(Firework(AnalyzeStressStrainData(structure=struct_si, db_file=db_file),
                    name="Analyze Elastic Data", parents=fws[1:],
                    spec={"_allow_fizzled_parents": True}))

wfname = "{}:{}".format(struct_si.composition.reduced_formula, "elastic constants")

wf = Workflow(fws, name=wfname)

# Powerups

In [8]:
from matmethods.vasp.vasp_powerups import add_modify_incar
from matmethods.vasp.workflows.presets.core import add_common_powerups

wf = add_modify_incar(wf, modify_incar_params={"incar_update": {"ENCUT": 700, "EDIFF": 1e-6}})
c = {"VASP_CMD": vasp_cmd, "DB_FILE": db_file, "reciprocal_density": 600}
wf = add_common_powerups(wf, c)

#wf.as_dict()["fws"][25]

# Emulate VASP

In [9]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

reference_dir = os.path.abspath(os.path.join("Si_elastic_runs"))

si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si-structure_optimization")}
for i, deformation in enumerate(deformations):
    si_ref_dirs["elastic_deformation_"+str(i+1)] = os.path.join(reference_dir, "Si-elastic_deformation-"+str(i+1))
    
wf_fake = use_fake_vasp(wf, si_ref_dirs, params_to_check=["ENCUT"])

#wf_fake.as_dict()["fws"][25]

# Connect to Launchpad and reset

In [10]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

2016-08-04 11:42:30,997 INFO Performing db tune-up
2016-08-04 11:42:31,004 INFO LaunchPad was RESET.


# Submit Workflow

In [11]:
lp.add_wf(wf_fake)

2016-08-04 11:42:34,543 INFO Added a workflow. id_map: {-1: 26, -26: 1, -25: 2, -24: 3, -23: 4, -22: 5, -21: 6, -20: 7, -19: 8, -18: 9, -17: 10, -16: 11, -15: 12, -14: 13, -13: 14, -12: 15, -11: 16, -10: 17, -9: 18, -8: 19, -7: 20, -6: 21, -5: 22, -4: 23, -3: 24, -2: 25}


{-26: 1,
 -25: 2,
 -24: 3,
 -23: 4,
 -22: 5,
 -21: 6,
 -20: 7,
 -19: 8,
 -18: 9,
 -17: 10,
 -16: 11,
 -15: 12,
 -14: 13,
 -13: 14,
 -12: 15,
 -11: 16,
 -10: 17,
 -9: 18,
 -8: 19,
 -7: 20,
 -6: 21,
 -5: 22,
 -4: 23,
 -3: 24,
 -2: 25,
 -1: 26}

In [12]:
def get_status():
    for id in lp.get_fw_ids():
        fw = lp.get_fw_by_id(id)
        print "Name:{}, State:{}".format(fw.name, fw.state)

def set_launch_dir(launch_dir):
    os.mkdir(launch_dir)
    os.chdir(launch_dir)
    
print len(wf_fake.as_dict()["fws"])

get_status()

26
Name:Analyze Elastic Data, State:WAITING
Name:Si-elastic_deformation_24, State:WAITING
Name:Si-elastic_deformation_23, State:WAITING
Name:Si-elastic_deformation_22, State:WAITING
Name:Si-elastic_deformation_21, State:WAITING
Name:Si-elastic_deformation_20, State:WAITING
Name:Si-elastic_deformation_19, State:WAITING
Name:Si-elastic_deformation_18, State:WAITING
Name:Si-elastic_deformation_17, State:WAITING
Name:Si-elastic_deformation_16, State:WAITING
Name:Si-elastic_deformation_15, State:WAITING
Name:Si-elastic_deformation_14, State:WAITING
Name:Si-elastic_deformation_13, State:WAITING
Name:Si-elastic_deformation_12, State:WAITING
Name:Si-elastic_deformation_11, State:WAITING
Name:Si-elastic_deformation_10, State:WAITING
Name:Si-elastic_deformation_9, State:WAITING
Name:Si-elastic_deformation_8, State:WAITING
Name:Si-elastic_deformation_7, State:WAITING
Name:Si-elastic_deformation_6, State:WAITING
Name:Si-elastic_deformation_5, State:WAITING
Name:Si-elastic_deformation_4, State:WAIT

# Launch structure optimization firework

In [14]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire, launch_rocket

db_file = os.path.join(os.getcwd(), "db.json")
#rapidfire(lp, fworker=FWorker(env={"db_file": db_file}))

set_launch_dir("structure _optimization")
launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-04 11:43:08,703 INFO Launching Rocket
2016-08-04 11:43:08,940 INFO RUNNING fw_id: 26 in directory: /home/ray/Software/workshop-2016/matmethods/structure _optimization
2016-08-04 11:43:08,955 INFO Task started: FileWriteTask.
2016-08-04 11:43:08,958 INFO Task completed: FileWriteTask 
2016-08-04 11:43:08,958 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2016-08-04 11:43:08,973 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2016-08-04 11:43:08,974 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.ModifyIncar}}.
2016-08-04 11:43:08,977 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.ModifyIncar}} 
2016-08-04 11:43:08,978 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-04 11:43:09,017 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-04 11:43:09,084 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, 

True

# Rapidfire structure deformation fireworks

In [15]:
#set_launch_dir("../def_test")
#launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))
rapidfire(lp, fworker=FWorker(env={"db_file": db_file}), nlaunches=24)

2016-08-04 11:43:26,174 INFO Created new dir /home/ray/Software/workshop-2016/matmethods/structure _optimization/launcher_2016-08-04-18-43-26-174635
2016-08-04 11:43:26,175 INFO Launching Rocket
2016-08-04 11:43:26,485 INFO RUNNING fw_id: 25 in directory: /home/ray/Software/workshop-2016/matmethods/structure _optimization/launcher_2016-08-04-18-43-26-174635
2016-08-04 11:43:26,490 INFO Task started: FileWriteTask.
2016-08-04 11:43:26,492 INFO Task completed: FileWriteTask 
2016-08-04 11:43:26,493 INFO Task started: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}}.
2016-08-04 11:43:26,690 INFO Task completed: {{matmethods.vasp.firetasks.glue_tasks.CopyVaspOutputs}} 
2016-08-04 11:43:26,691 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}}.
2016-08-04 11:43:26,714 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteTransmutedStructureIOSet}} 
2016-08-04 11:43:26,715 INFO Task started: {{matmethods.vasp.firetasks.write_input

In [16]:
for id in lp.get_fw_ids():
    fw = lp.get_fw_by_id(id)
    print fw.name, fw.fw_name, fw.state

Analyze Elastic Data fireworks::Firework READY
Si-elastic_deformation_24 fireworks::Firework COMPLETED
Si-elastic_deformation_23 fireworks::Firework COMPLETED
Si-elastic_deformation_22 fireworks::Firework COMPLETED
Si-elastic_deformation_21 fireworks::Firework COMPLETED
Si-elastic_deformation_20 fireworks::Firework COMPLETED
Si-elastic_deformation_19 fireworks::Firework COMPLETED
Si-elastic_deformation_18 fireworks::Firework COMPLETED
Si-elastic_deformation_17 fireworks::Firework COMPLETED
Si-elastic_deformation_16 fireworks::Firework COMPLETED
Si-elastic_deformation_15 fireworks::Firework COMPLETED
Si-elastic_deformation_14 fireworks::Firework COMPLETED
Si-elastic_deformation_13 fireworks::Firework COMPLETED
Si-elastic_deformation_12 fireworks::Firework COMPLETED
Si-elastic_deformation_11 fireworks::Firework COMPLETED
Si-elastic_deformation_10 fireworks::Firework COMPLETED
Si-elastic_deformation_9 fireworks::Firework COMPLETED
Si-elastic_deformation_8 fireworks::Firework COMPLETED
Si-

# Launch stress-strain analyzer firework

In [17]:
launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-04 11:46:02,552 INFO Launching Rocket
2016-08-04 11:46:02,826 INFO RUNNING fw_id: 1 in directory: /home/ray/Software/workshop-2016/matmethods/structure _optimization
2016-08-04 11:46:02,838 INFO Task started: FileWriteTask.
2016-08-04 11:46:02,841 INFO Task completed: FileWriteTask 
2016-08-04 11:46:02,841 INFO Task started: {{matmethods.vasp.workflows.base.elastic.AnalyzeStressStrainData}}.
2016-08-04 11:46:02,842 INFO matmethods.vasp.workflows.base.elastic PARSING INITIAL OPTIMIZATION DIRECTORY: /home/ray/Software/workshop-2016/matmethods/structure _optimization
2016-08-04 11:46:02,843 INFO matmethods.vasp.drones Getting task doc for base dir :/home/ray/Software/workshop-2016/matmethods/structure _optimization
2016-08-04 11:46:03,543 INFO matmethods.vasp.drones Post-processing dir:/home/ray/Software/workshop-2016/matmethods/structure _optimization
2016-08-04 11:46:03,545 WARNING matmethods.vasp.drones Transformations file does not exist.
2016-08-04 11:46:03,546 INFO matmethod

/home/ray/Software/matproj/pymatgen/pymatgen/analysis/elasticity/elastic.py:61: UserWarning: Input elasticity tensor does not satisfy standard symmetries
  warnings.warn("Input elasticity tensor does "
Traceback (most recent call last):
  File "/home/ray/Software/matproj/fireworks/fireworks/core/rocket.py", line 211, in run
    m_action = t.run_task(my_spec)
  File "/home/ray/Software/matproj/MatMethods/matmethods/vasp/workflows/base/elastic.py", line 102, in run_task
    kg_average = result.kg_average
  File "/home/ray/Software/matproj/pymatgen/pymatgen/analysis/elasticity/elastic.py", line 166, in kg_average
    return [self.k_voigt, self.g_voigt, self.k_reuss, self.g_reuss,
  File "/home/ray/Software/matproj/pymatgen/pymatgen/analysis/elasticity/elastic.py", line 135, in k_reuss
    return 1. / self.compliance_tensor[:3, :3].sum()
  File "/home/ray/Software/matproj/pymatgen/pymatgen/analysis/elasticity/elastic.py", line 112, in compliance_tensor
    return np.linalg.inv(self.voigt)


2016-08-04 11:46:03,850 INFO Rocket finished


True

# Retrieve and Verify

In [18]:
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db[creds["collection"]]

In [22]:
from pymongo import DESCENDING

d1 = get_task_collection("../db.json").find_one({"task_label": "Analyze Elastic Data"}, sort=[("_id", DESCENDING)])